Copy the file **spy.2008.2021.csv.gz** and **spy.csv.gz** located in https://github.com/crapher/medium/tree/main/13.TrendIntraday\data to your colab folder

In [1]:
import os
import math
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.utils import set_random_seed
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
# Constants
BARS=15
RESULT_DIR='./result'
DATE_SPLIT='2019-06-01'

In [3]:
# Get the data and generate the train, validation, and test datasets
def get_datasets():

    train_val = pd.read_csv('./spy.2008.2021.csv.gz', compression='gzip')
    train_val = train_val[['date','open','close']]
    train_val['date'] = pd.to_datetime(train_val['date'])

    test = pd.read_csv('./spy.csv.gz', compression='gzip')
    test = test[['date','open','close']]
    test['date'] = pd.to_datetime(test['date'])

    train = train_val[train_val['date'] <= DATE_SPLIT]
    validation = train_val[(train_val['date'] > DATE_SPLIT) & (train_val['date'] < test['date'].min())]

    return train, validation, test

In [4]:
# Process Dataframe and return features and targets
def get_features_targets(df, scale_obs=True):

    feature_result = []
    dates = []

    # Remove duplicated dates
    df = df.groupby(by='date').mean().reset_index()

    # Get Features based on BARS configuration
    features = df[((df['date'].dt.hour == 9) & (df['date'].dt.minute >= 30)) &
                   (df['date'].dt.hour == 9) & (df['date'].dt.minute < 30 + BARS)]
    features = features.groupby(features['date'].dt.date)

    for dt, feature in features:

        if len(feature) != BARS:
            feature = feature.set_index('date')
            feature = feature.resample('1T').asfreq().reindex(pd.date_range(str(dt) + ' 09:30:00', str(dt) + f' 09:{30+BARS-1}:00', freq='1T'))
            feature = feature.reset_index()
            feature['close'] = feature['close'].fillna(method='ffill')
            feature['open'] = feature['open'].fillna(feature['close'])
            feature = feature.dropna()

        if len(feature) == BARS:
            feature = feature['close'].values

            if scale_obs:
                feature -= np.min(feature)
                feature /= np.max(np.abs(feature))
                feature = np.nan_to_num(feature, nan=0.0, posinf=0.0, neginf=0.0)

            feature_result.append(feature)
            dates.append(dt)

    # Get Targets Trend based on first and last value / day (0: DOWN - 1: UP)
    targets = df.set_index('date')
    targets = targets.resample('1D').agg({'open':'first', 'close':'last'})
    targets = targets.loc[dates].reset_index().sort_values(by='date')
    targets['trend'] = np.where(targets['open'] < targets['close'], 1, 0)

    print(len(feature_result), len(targets))
    return np.array(feature_result), np.array(targets['trend'].values)

In [5]:
# Show the result of the operation
def show_result(target, pred, ds_type='TEST'):

    target = np.array(target)
    pred = np.array(pred)

    print(f' RESULT {ds_type.upper()} '.center(56, '*'))

    print('* Confusion Matrix (Top: Predicted - Left: Real)')
    print(confusion_matrix(y_true=target, y_pred=pred))

    print('* Classification Report')
    print(classification_report(target, pred))

In [6]:
# Save the results
def save_result(target, pred, name):

    os.makedirs(RESULT_DIR, exist_ok=True)
    df = pd.DataFrame.from_dict({'pred': pred, 'target': target})
    df.to_csv(f'{RESULT_DIR}/{name}', index=False)

In [7]:
# Return the predicted values
def get_predicted_values(model, features):

    pred_y = model.predict(features)
    pred_y = pred_y.flatten()
    pred_y = np.where(pred_y > 0.5, 1, 0)
    return pred_y

In [8]:
# Get the datasets to be used in the tests
train, validation, test = get_datasets()

In [9]:
# Get Features and Targets
train_x, train_y = get_features_targets(train)
val_x, val_y = get_features_targets(validation)
test_x, test_y = get_features_targets(test)

2858 2858
487 487
529 529


In [10]:
# Train Model
set_random_seed(42)                         # Allow reproducibility
early_stopping = EarlyStopping(patience=2)  # Stop if the model does not improve

model = Sequential()
model.add(Dense(16, input_dim=15, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_x, train_y, epochs=100, batch_size=32, validation_data=(val_x, val_y), callbacks=[early_stopping])

Epoch 1/100
90/90 [==============================] - 1s 5ms/step - loss: 0.6832 - accuracy: 0.5675 - val_loss: 0.6743 - val_accuracy: 0.5852
Epoch 2/100
90/90 [==============================] - 0s 2ms/step - loss: 0.6646 - accuracy: 0.6043 - val_loss: 0.6727 - val_accuracy: 0.5975
Epoch 3/100
90/90 [==============================] - 0s 3ms/step - loss: 0.6599 - accuracy: 0.6057 - val_loss: 0.6699 - val_accuracy: 0.6057
Epoch 4/100
90/90 [==============================] - 0s 2ms/step - loss: 0.6582 - accuracy: 0.6123 - val_loss: 0.6682 - val_accuracy: 0.5955
Epoch 5/100
90/90 [==============================] - 0s 3ms/step - loss: 0.6569 - accuracy: 0.6106 - val_loss: 0.6666 - val_accuracy: 0.6078
Epoch 6/100
90/90 [==============================] - 0s 2ms/step - loss: 0.6557 - accuracy: 0.6148 - val_loss: 0.6657 - val_accuracy: 0.6037
Epoch 7/100
90/90 [==============================] - 0s 2ms/step - loss: 0.6547 - accuracy: 0.6137 - val_loss: 0.6653 - val_accuracy: 0.5996
Epoch 8/100
9

In [11]:
# Predict and save train values
pred_y = get_predicted_values(model, train_x)
save_result(train_y, pred_y, 'nn.train.csv.gz')

90/90 [==============================] - 0s 1ms/step


In [12]:
# Predict and save validation values
pred_y = get_predicted_values(model, val_x)
save_result(val_y, pred_y, 'nn.val.csv.gz')

16/16 [==============================] - 0s 1ms/step


In [13]:
# Predict, show and save test values
pred_y = get_predicted_values(model, test_x)
show_result(test_y, pred_y)
save_result(test_y, pred_y, 'nn.test.csv.gz')

17/17 [==============================] - 0s 1ms/step
********************* RESULT TEST **********************
* Confusion Matrix (Top: Predicted - Left: Real)
[[130 119]
 [ 81 199]]
* Classification Report
              precision    recall  f1-score   support

           0       0.62      0.52      0.57       249
           1       0.63      0.71      0.67       280

    accuracy                           0.62       529
   macro avg       0.62      0.62      0.62       529
weighted avg       0.62      0.62      0.62       529

